In [ ]:
import pandas as pd
import plotly.express as px
from IPython import embed
pd.set_option('use_inf_as_na', True)

In [ ]:
def loadDataUS(baseURL, type):

  if type=='cases':
    dropping = ['Lat', 'Long_', 'Combined_Key', 'UID',	'iso2',	'iso3',	'code3', 'FIPS', 'Admin2', 'Country_Region']
    filename = 'time_series_covid19_confirmed_US.csv'
    columnName = 'Confirmed'
  else:
    dropping = ['Lat', 'Long_', 'Combined_Key', 'UID',	'iso2',	'iso3',	'code3', 'FIPS', 'Admin2', 'Country_Region', 'Population']
    filename = 'time_series_covid19_deaths_US.csv'    
    columnName = 'Deaths'

  data = pd.read_csv(baseURL+filename) \
    .drop(dropping, axis=1) 
  data = data.groupby(['Province_State']).sum().reset_index()
  data = data.melt(id_vars=['Province_State'], var_name='date', value_name=columnName)
  data = data.astype({'date':'datetime64[ns]', columnName:'Int64'}, errors='ignore')
  data['Province_State'].fillna('<all>', inplace=True)
  data[columnName].fillna(0, inplace=True)
  return data

def meltit(df):
  df.fillna(0, inplace=True)
  df = df.reset_index()
  df = df.melt(id_vars=['date'], value_vars = states, var_name=['Province_State'], value_name='Confirmed')
  return df


In [ ]:
baseURL = "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/"
df = loadDataUS(baseURL, 'cases')
df = df.pivot_table(values='Confirmed', index='date', columns='Province_State')
states = list(df.keys()[1:])

dfraw = meltit(df)

dfmean = df.diff().rolling(window=5,center=False).mean()
dfmean = meltit(dfmean)
fig = px.line(dfmean, x="date", y="Confirmed", color="Province_State", log_y=False)
fig.show()

dfgrowth = df.pct_change()
dfgrowth = meltit(dfgrowth)
fig = px.line(dfgrowth, x="date", y="Confirmed", color="Province_State", log_y=False)
fig.show()

dfraw['mean_last_5_days'] = dfmean.Confirmed
fig = px.line(dfraw, x="Confirmed", y="mean_last_5_days", color="Province_State", log_x=True, log_y=True, labels=dict(Confirmed='Total Confirmed Cases', mean_last_5_days='Mean Daily New Cases from Last 5 Days'))
fig.show()
fig.write_html('states.html')

df.to_csv('confirmed_us.csv')
fig = px.line(x=dfmean.Confirmed, y=dfraw.Confirmed, log_y=False)

